<a href="https://colab.research.google.com/github/jeffblackadar/CRANE-CCAD-maps/blob/main/maps_ocr_google_cloud_vision_1_tile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read Arabic text on a map.
## Uses Google Coud Vision
### Requires a Google Cloud Vision project with billing enabled
https://cloud.google.com/vision/docs/setup

### Quick start:
https://cloud.google.com/vision/docs/ocr

In [ ]:
!pip install --upgrade google-cloud-vision
!pip install arabic_reshaper
!pip install bidi
!pip install python-bidi

In [8]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/william_white/map-ocr-project-e790a806af19.json'

In [9]:
# Code adapted from https://cloud.google.com/vision/docs/ocr  (Thanks for this)
from google.cloud import vision
import io
def detect_text(path):
    lines_of_text = []  
    """Detects text in the file."""

    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(
    image=image,
    image_context={"language_hints": ["ar"]},  
     )
    
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        line_data = []  
        ocr_text = text.description
        print('\n"{}"'.format(text.description))

        #Don't want just numbers
        if(not ocr_text.isnumeric()):
            
            line_data.append(text.description)
        
            # store the vertices as a list  x0,y0,x1,y1, etc.
            points = []
            for vertex in text.bounding_poly.vertices:
                points.append(vertex.x)
                points.append(vertex.y)
            line_data.append(points)
            lines_of_text.append(line_data)

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    
    if(len(lines_of_text)>1):
        #It seems the first entry back from Google contains all the text (multiple lines), I just want each line.
        lines_of_text.pop(0)
    return(lines_of_text)
lines_of_text = detect_text('/content/drive/MyDrive/crane_maps_syria/maps_large/Djeble_georef/r07c09.jpg')

Texts:

"حميميم
384
2
42
فعنا
تل المفروم
38
"

"حميميم"

"384"

"2"

"42"

"فعنا"

"تل"

"المفروم"

"38"


In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import ImageFont, ImageDraw, Image
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np

# This is the 1 map tile we'll use for this example:
img = cv2.imread('/content/drive/MyDrive/crane_maps_syria/maps_large/Djeble_georef/r07c09.jpg',-1)

# Thanks: https://stackoverflow.com/questions/59896297/how-to-draw-arabic-text-on-the-image-using-cv2-puttextcorrectly-pythonopenc

fontpath = "/content/drive/MyDrive/crane_font/arial.ttf" # <== https://www.freefontspro.com/14454/arial.ttf  
font_pil = ImageFont.truetype(fontpath, 32)
font                   = cv2.FONT_HERSHEY_SIMPLEX
fontColor              = (200,0,0)

img_pil = Image.fromarray(img)
draw = ImageDraw.Draw(img_pil)

for l in lines_of_text:
    print(l[0])
    pts = l[1]
    #This is needed to handle the Arabic text
    reshaped_text = arabic_reshaper.reshape(l[0])
    bidi_text = get_display(reshaped_text)
    draw.text((int(pts[0]), int(pts[1])),bidi_text, font = font_pil,fill=(255,0,0,255))
     
img2 = np.array(img_pil)
for l in lines_of_text:
    pts = l[1]
    cv2.rectangle(img2,(int(pts[0]),int(pts[1])),(int(pts[4]),int(pts[5])),fontColor,1) 

from matplotlib.pyplot import figure
figure(num=None, figsize=(20, 20), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(img2)